# Introduction

In herer I will construct my own XGBoost classification model. I will be presenting a business scenario and a dataset to build on the skills and techniques I learned in the Decision Tree and Random Forest lessons. I will practice training, tuning, and evaluation an XGBoost model and compare its performance to previous models' to decide which model is best. 

**Data Dictionary**

This activity uses a dataset called 
*Invistico_Airline.csv*
. It represents the details of customer feedback for Invistico Airlines (a fictional name). 

**The dataset contains**: 23 columns of customer feedback, each row representing one customer’s responses.

**The dataset contains**:

129,880 rows – each row is a different customer response 

23 columns

# Step 1: Imports

**Import packages**

Import *pandas, numpy*, and *matplotlib* for data preparation. Import *scikit-learn* (sklearn) for model preparation and evaluation. And, import *xgboost*, which provides the classification algorithm to implement to formulate of my predictive model.

In [1]:
# Import relevant libraries and modules.

import numpy as np
import pandas as pd

# This is the classifier
from xgboost import XGBClassifier

# This is the function that helps plot feature importance 
from xgboost import plot_importance

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score,\
f1_score, confusion_matrix, ConfusionMatrixDisplay, RocCurveDisplay

import matplotlib.pyplot as plt

# This displays all of the columns, preventing Juptyer from redacting them.
pd.set_option('display.max_columns', None)

# This module lets us save our models once we fit them.
import pickle

**Load the dataset**

To formulate your model, *pandas* is used to import a csv of airline passenger satisfaction data called *Invistico_Airline.csv*. This DataFrame is called *airline_data*.

In [2]:
airline_data = pd.read_csv('/kaggle/input/aviation-satisfaction-data/Invistico_Airline.csv')

**Display the data**

Examine the first 10 rows of data.

In [3]:
airline_data.head(10)

,satisfaction,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,Gate location,Inflight wifi service,Inflight entertainment,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes
0,satisfied,Loyal Customer,65,Personal Travel,Eco,265,0,0,0,2,2,4,2,3,3,0,3,5,3,2,0,0.0
1,satisfied,Loyal Customer,47,Personal Travel,Business,2464,0,0,0,3,0,2,2,3,4,4,4,2,3,2,310,305.0
2,satisfied,Loyal Customer,15,Personal Travel,Eco,2138,0,0,0,3,2,0,2,2,3,3,4,4,4,2,0,0.0
3,satisfied,Loyal Customer,60,Personal Travel,Eco,623,0,0,0,3,3,4,3,1,1,0,1,4,1,3,0,0.0
4,satisfied,Loyal Customer,70,Personal Travel,Eco,354,0,0,0,3,4,3,4,2,2,0,2,4,2,5,0,0.0
5,satisfied,Loyal Customer,30,Personal Travel,Eco,1894,0,0,0,3,2,0,2,2,5,4,5,5,4,2,0,0.0
6,satisfied,Loyal Customer,66,Personal Travel,Eco,227,0,0,0,3,2,5,5,5,5,0,5,5,5,3,17,15.0
7,satisfied,Loyal Customer,10,Personal Travel,Eco,1812,0,0,0,3,2,0,2,2,3,3,4,5,4,2,0,0.0
8,satisfied,Loyal Customer,56,Personal Travel,Business,73,0,0,0,3,5,3,5,4,4,0,1,5,4,4,0,0.0
9,satisfied,Loyal Customer,22,Personal Travel,Eco,1556,0,0,0,3,2,0,2,2,2,4,5,3,4,2,30,26.0


**Display the data type for each column**

Observe the types of data present within this dataset.

In [4]:
airline_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129880 entries, 0 to 129879
Data columns (total 22 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   satisfaction                       129880 non-null  object 
 1   Customer Type                      129880 non-null  object 
 2   Age                                129880 non-null  int64  
 3   Type of Travel                     129880 non-null  object 
 4   Class                              129880 non-null  object 
 5   Flight Distance                    129880 non-null  int64  
 6   Seat comfort                       129880 non-null  int64  
 7   Departure/Arrival time convenient  129880 non-null  int64  
 8   Food and drink                     129880 non-null  int64  
 9   Gate location                      129880 non-null  int64  
 10  Inflight wifi service              129880 non-null  int64  
 11  Inflight entertainment             1298

The target (or predicted) variable for passenger satisfaction is **satisfaction**.

Here are my initial hypotheses about which variables will be valuable in predicting satisfaction:

* **Customer Type:** The type of customer (e.g., loyalty member, first-time flyer) may influence satisfaction.
* **Class:** The class of service (e.g., economy, business, first) may affect satisfaction.
* **Flight Distance:** Longer flights might lead to higher expectations and potentially lower satisfaction if not met.
* **Seat comfort:** A comfortable seat is likely to contribute to higher satisfaction.
* **Departure/Arrival time convenient:** If the flight times are convenient for the customer, they may be more satisfied.
* **Food and drink:** The quality and variety of food and drink options could impact satisfaction.
* **Inflight wifi service, Inflight entertainment, Online support:** These amenities could influence satisfaction, especially for long flights.
* **On-board service, Leg room service, Baggage handling, Checkin service:** Good service in these areas could enhance satisfaction.
* **Cleanliness:** A clean aircraft is likely to contribute to higher satisfaction.
* **Online boarding:** A smooth online boarding experience could improve satisfaction.
* **Departure Delay in Minutes, Arrival Delay in Minutes:** Delays could negatively impact satisfaction.

It's important to note that these are just initial hypotheses, and the actual importance of each variable in predicting satisfaction will need to be determined through model training and evaluation.


# Step 2: Model preparation

**Prepare data for predictions**

In [5]:
# Convert the object predictor variables to numerical dummies.

airline_data_dummies = pd.get_dummies(airline_data, 
                                         columns=['satisfaction','Customer Type','Type of Travel','Class'])

**Isolate your target and predictor variables**

Separately define the target variable (*satisfaction*) and the features.

In [6]:
# Define the y (target) variable.

y = airline_data_dummies['satisfaction_satisfied']

# Define the X (predictor) variables.

X = airline_data_dummies.drop(['satisfaction_satisfied','satisfaction_dissatisfied'], axis = 1)

**Divide your data**

Divide data into a training set (75% of the data) and test set (25% of the data). This is an important step in the process, as it allows me to reserve a part of the data that the model has not used to test how well the model generalizes (or performs) on new data.

In [7]:
# Perform the split operation on your data.
# Assign the outputs as follows: X_train, X_test, y_train, y_test.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, 
                                                    stratify=y, random_state=42)

# Step 3: Model building

**"Instantiate" your XGBClassifer**

Before I fit my model to my airline dataset, first I need to create the XGB Classifier model and define its objective. I'll use this model to fit and score different hyperparameters during the GridSearch cross-validation process.

In [8]:
# Define xgb to be your XGBClassifier.

xgb = XGBClassifier(objective='binary:logistic', random_state=0)

**Define the parameters for hyperparameter tuning**

To identify suitable parameters for your xgboost model, first define the parameters for hyperparameter tuning. Specifically, consider tuning max_depth, min_child_weight, learning_rate, n_estimators, subsample, and/or colsample_bytree

In [9]:
# Define parameters for tuning as `cv_params`.

cv_params = {'max_depth': [4],
                'min_child_weight': [3],
                'learning_rate': [0.1],
                'n_estimators': [5],
                'subsample': [0.7],
                'colsample_bytree': [0.7]
            }

**Define how the models will be evaluated**

Define how the models will be evaluated for hyperparameter tuning. To yield the best understanding of model performance, utilize a suite of metrics.

In [10]:
# Define your criteria as `scoring`.

scoring = {'accuracy', 'precision', 'recall', 'f1'}

**Construct the GridSearch cross-validation**

Construct the GridSearch cross-validation using the model, parameters, and scoring metrics I defined. Additionally, define the number of folds and specify which metric from above will guide the refit strategy.

In [11]:
# Construct your GridSearch.

xgb_cv = GridSearchCV(xgb, cv_params, scoring=scoring, cv=5, refit='f1')

**Fit the GridSearch model to your training data**

In [12]:
%%time
# fit the GridSearch model to training data

xgb_cv.fit(X_train, y_train)

CPU times: user 7.16 s, sys: 58.4 ms, total: 7.22 s
Wall time: 3.49 s


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=0, ...),
             param_grid={'colsample_bytree': [0.7], 'learning_rate': [0.1],
                         'max_depth': [4], 'min_child_weight': [3],
                         'n_estimators': [5], 'subsample': [0.7]},
             refit='f1', scoring={'accuracy', 'recall', 'f1', 'precision'})

The optimal set of parameters yielded by the GridSearchCV is:

* **colsample_bytree**: 0.7
* **learning_rate**: 0.1
* **max_depth**: 4
* **min_child_weight**: 3
* **n_estimators**: 5
* **subsample**: 0.7

This set of parameters was likely chosen based on the scoring metric specified in the `scoring` parameter of GridSearchCV, which in this case is a dictionary containing 'accuracy', 'recall', 'f1', and 'precision'. The `refit` parameter set to 'f1' indicates that the final model will be refitted using the parameters that achieved the highest F1-score.


**Save your model for reference using pickle**

Use the *pickle* library to save the output of this model.

In [13]:
# Use `pickle` to save the trained model.

with open('/kaggle/working/' + 'xgb_cv_model.pickle', 'wb') as to_write:
    pickle.dump(xgb_cv, to_write) 

# Step 4: Results and evaluation

**Formulate predictions on my test set**

To evaluate the predictions yielded from my model, leverage a series of metrics and evaluation techniques from scikit-learn by examining the actual observed values in the test set relative to my model's prediction.